# HHA550_Diabetes Prediction Dataset

## Healtcare-dataset-stroke-data

#### (Check Modules folders for csv and ipynb for each class)

# DATA
## Diabetes Prediction Dataset


# .CSV Data
### cleaned_diabetic_data.csv

# IMPORTING Everthing

In [1]:
# Commands to install some of the libraries in-case if they are not installed
# Any other library that needs to be installed just use: !pip install <library name>
# !pip install seaborn
# !pip install missingno
# !pip install xgboost
# !pip install catboost
# !pip install regex
# !pip install sklearn
# !pip install pandas
# !pip install numpy
# !pip install imblearn
# !pip install lightgbm

In [1]:
import pandas as pd   # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np   # linear algebra
import matplotlib.pyplot as plt  #graphs and plots
import seaborn as sns   #data visualizations 
import csv # Some extra functionalities for csv  files - reading it as a dictionary
from lightgbm import LGBMClassifier #sklearn is for machine learning and statistical modeling including classification, regression, clustering and dimensionality reduction 

from sklearn.model_selection import train_test_split, cross_validate   #break up dataset into train and test sets

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler


# importing python library for working with missing data
import missingno as msno
# To install missingno use: !pip install missingno
import re    # This library is used to perform regex pattern matching

# import various functions from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, classification_report, make_scorer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

Import additional items as needed...
We may not use them all in this course...

In [39]:
from sklearn.model_selection import KFold,cross_val_score, RepeatedStratifiedKFold,StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.compose import make_column_transformer
from imblearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score,\
                            precision_score, recall_score, roc_auc_score,\
                            ConfusionMatrixDisplay, classification_report, RocCurveDisplay, f1_score
from sklearn.linear_model import LinearRegression
import plotly 
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import warnings
warnings.filterwarnings("ignore")

* If from imblearn.oversampling import SMOTE does not load use
    `conda install -c conda-forge imbalanced-learn`
* Then rerun
    `from imblearn.over_sampling import SMOTE`

# Exploratory Data Analysis (EDA)

## Start with Loading the CSV Data

In [46]:
dm = pd.read_csv('/Users/lozo/Developer/AHI_Github/HHA550_Analysis/data/cleaned_diabetic_data.csv')
dm_analysis = dm[['readmitted', 'age', 'weight', 'A1Cresult']]

# Breaking the data up into Train & Test

In [21]:
train_df, valid_df, test_df = np.split(dm_analysis.sample(frac=1, random_state=42), 
                                        [int(.7*len(dm_analysis)), int(0.85*len(dm_analysis))])
train_df = train_df.reset_index(drop = True)
valid_df = valid_df.reset_index(drop = True)
test_df = test_df.reset_index(drop = True)

In [22]:
dm_analysis.readmitted.value_counts()

readmitted
0    1610
1    1449
Name: count, dtype: int64

In [23]:
train_df.readmitted.value_counts()

readmitted
0    1108
1    1033
Name: count, dtype: int64

In [24]:
valid_df.readmitted.value_counts()

readmitted
0    268
1    191
Name: count, dtype: int64

In [25]:
test_df.readmitted.value_counts()

readmitted
0    234
1    225
Name: count, dtype: int64

# Treating the Imbalance in the Data

Imbalance in the data means that one of the classes in the data is too less as compared to the others. Typically, it is better to balance the data in some way to give the positives more weight. There are 3 strategies that are typically utilized:

* Sub-sample the more dominant class: use a random subset of the negatives
* Over-sample the imbalanced class: use the same positive samples multiple times
* Create synthetic positive data

Usually, you will want to use the latter two methods if you only have a handful of positive cases. Since we have a few thousand positive cases, let's use the sub-sample approach. Here, we will create a balanced training data set that has 50% positive and 50% negative. You can also play with this ratio to see if you can get an improvement.

In [27]:
def calc_prevalence(y_actual):
    
    '''
    This function is to understand the ratio/distribution of the classes that we are going to predict for.
    
    Params:
    1. y_actual: The target feature
    
    Return:
    1. (sum(y_actual)/len(y_actual)): The ratio of the postive class in the comlpete data.
    '''
    
    return (sum(y_actual)/len(y_actual))

In [29]:
# split the training data into positive and negative
rows_pos = train_df.readmitted == 1
df_train_pos = train_df.loc[rows_pos]
df_train_neg = train_df.loc[~rows_pos]

# merge the balanced data
dm_df_balanced = pd.concat([df_train_pos, df_train_neg.sample(n = len(df_train_pos), random_state = 111)],axis = 0)

# shuffle the order of training samples 
dm_df_balanced = dm_df_balanced.sample(n = len(dm_df_balanced), random_state = 42).reset_index(drop = True)

print('Train balanced prevalence(n = %d):%.3f'%(len(dm_df_balanced), \
                                                calc_prevalence(dm_df_balanced.readmitted.values)))

Train balanced prevalence(n = 2066):0.500


In [31]:
dm_df_balanced.readmitted.value_counts()

readmitted
0    1033
1    1033
Name: count, dtype: int64

In [33]:
X_train = dm_df_balanced.drop('readmitted',axis=1)

y_train = dm_df_balanced['readmitted']

X_valid = valid_df.drop('readmitted',axis=1)

y_valid = valid_df['readmitted']

X_test = test_df.drop('readmitted',axis=1)

y_test = test_df['readmitted']

In [34]:
scaler=StandardScaler()
X_train[['age', 'weight', 'A1Cresult']] = pd.DataFrame(scaler.fit_transform(X_train[['age', 'weight', 'A1Cresult']]),columns=['age', 'weight', 'A1Cresult'])
X_valid[['age', 'weight', 'A1Cresult']] = pd.DataFrame(scaler.transform(X_valid[['age', 'weight', 'A1Cresult']]),columns=['age', 'weight', 'A1Cresult'])
X_test[['age', 'weight', 'A1Cresult']] = pd.DataFrame(scaler.transform(X_test[['age', 'weight', 'A1Cresult']]),columns=['age', 'weight', 'A1Cresult'])

# Creating and Understanding Models

In [35]:
def calc_specificity(y_actual, y_pred, thresh):
    # calculates specificity
    return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)

def print_report(y_actual, y_pred, thresh = 0.5):
    
    '''
    This function calculates all the metrics to asses the machine learning models.
    
    Params:
    1. y_actual: The actual values for the target variable.
    2. y_pred: The predicted values for the target variable.
    3. thresh: The threshold for the probability to be considered as a positive class. Default value 0.5
    
    Return:
    1. AUC
    2. Accuracy
    3. Recall
    4. Precision
    5. Specificity
    '''
    
    auc = roc_auc_score(y_actual, y_pred)
    accuracy = accuracy_score(y_actual, (y_pred > thresh))
    recall = recall_score(y_actual, (y_pred > thresh))
    precision = precision_score(y_actual, (y_pred > thresh))
    specificity = calc_specificity(y_actual, y_pred, thresh)
    print('AUC:%.3f'%auc)
    print('accuracy:%.3f'%accuracy)
    print('recall:%.3f'%recall)
    print('precision:%.3f'%precision)
    print('specificity:%.3f'%specificity)
    print('prevalence:%.3f'%calc_prevalence(y_actual))
    print(' ')
    return auc, accuracy, recall, precision, specificity

## Linear Regression

In [40]:
lnr = LinearRegression()
lnr.fit(X_train, y_train)

y_valid_preds = lnr.predict(X_valid)

In [41]:
y_valid_preds

array([0.51444151, 0.49206856, 0.5031647 , 0.49200833, 0.51444151,
       0.50322492, 0.53639287, 0.50322492, 0.5031647 , 0.40892619,
       0.51438129, 0.50328515, 0.49200833, 0.50322492, 0.50322492,
       0.51438129, 0.51438129, 0.50322492, 0.50328515, 0.38673392,
       0.50310447, 0.48055083, 0.5142006 , 0.49206856, 0.51426083,
       0.51432106, 0.49188788, 0.48097243, 0.49206856, 0.49212879,
       0.5031647 , 0.49200833, 0.49218902, 0.51426083, 0.50322492,
       0.49212879, 0.50322492, 0.49212879, 0.49218902, 0.50322492,
       0.49206856, 0.49206856, 0.49182765, 0.49206856, 0.49212879,
       0.49176742, 0.50322492, 0.5031647 , 0.50322492, 0.50328515,
       0.51432106, 0.54766969, 0.50322492, 0.50304424, 0.49212879,
       0.50328515, 0.50310447, 0.50322492, 0.51408015, 0.49206856,
       0.49206856, 0.51438129, 0.50334538, 0.50322492, 0.46963538,
       0.53663378, 0.46969561, 0.50310447, 0.49206856, 0.50322492,
       0.5031647 , 0.51444151, 0.49200833, 0.50304424, 0.48079

## Logistic Regression

In [42]:
lr=LogisticRegression(random_state = 42, solver = 'newton-cg', max_iter = 200)
lr.fit(X_train, y_train)

y_valid_preds = lr.predict_proba(X_valid)[:,1]

print('Metrics for Validation data:')

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,y_valid_preds, 0.5)

Metrics for Validation data:
AUC:0.520
accuracy:0.501
recall:0.654
precision:0.434
specificity:0.392
prevalence:0.416
 


## Explaining Results

In [43]:
from IPython import display
display.Image("C:/Users/kaden/Desktop/HHA550_Stroke_Data/Explaining_Results.png")

FileNotFoundError: No such file or directory: 'C:/Users/kaden/Desktop/HHA550_Stroke_Data/Explaining_Results.png'

FileNotFoundError: No such file or directory: 'C:/Users/kaden/Desktop/HHA550_Stroke_Data/Explaining_Results.png'

<IPython.core.display.Image object>

Lets look at some other models to see if we get better results.

## KNN Model

In [44]:
knn = KNeighborsClassifier(n_neighbors = 100)
knn.fit(X_train, y_train)

knn_preds = knn.predict_proba(X_valid)[:,1]

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,knn_preds, 0.5)

AUC:0.545
accuracy:0.534
recall:0.639
precision:0.457
specificity:0.440
prevalence:0.416
 


## Stochastic Gradient Descent Model

In [45]:
sgdc=SGDClassifier(loss = 'log',alpha = 0.1,random_state = 42)
sgdc.fit(X_train, y_train)

sgd_preds = sgdc.predict_proba(X_valid)[:,1]

print('Stochastic Gradient Descent')
print('Validation:')
sgdc_valid_auc, sgdc_valid_accuracy, sgdc_valid_recall, \
                sgdc_valid_precision, sgdc_valid_specificity = print_report(y_valid,sgd_preds, 0.5)

Stochastic Gradient Descent
Validation:
AUC:0.520
accuracy:0.501
recall:0.654
precision:0.434
specificity:0.392
prevalence:0.416
 


## Decision Tree

In [21]:
dc_clf = DecisionTreeClassifier(random_state=42, max_depth = 10)
dc_clf.fit(X_train, y_train)

dc_preds_proba = dc_clf.predict_proba(X_valid)[:,1]
dc_preds = dc_clf.predict(X_valid)

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,dc_preds_proba, 0.5)

AUC:0.641
accuracy:0.680
recall:0.600
precision:0.095
specificity:0.685
prevalence:0.052
 


## Random Forest

In [22]:
rf_clf = RandomForestClassifier(random_state=111, max_depth = 6)

rf_clf.fit(X_train, y_train)

rf_preds = rf_clf.predict(X_valid)
rf_preds_proba = rf_clf.predict_proba(X_valid)[:, 1]

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,rf_preds_proba, 0.5)

AUC:0.822
accuracy:0.719
recall:0.750
precision:0.128
specificity:0.718
prevalence:0.052
 


## Linear SVC

In [23]:
lsvc_clf = LinearSVC(random_state=111)
lsvc_clf.fit(X_train, y_train)

lsvc_preds = lsvc_clf.decision_function(X_valid)

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,lsvc_preds, 0.5)

AUC:0.820
accuracy:0.858
recall:0.450
precision:0.171
specificity:0.880
prevalence:0.052
 


## Gradient Boosting Model

In [24]:
gb_clf = GradientBoostingClassifier(n_estimators = 100, criterion='friedman_mse', learning_rate = 1.0, max_depth = 3,\
                                    random_state = 111)

gb_clf.fit(X_train, y_train)
gb_preds = gb_clf.predict(X_valid)
gb_preds_proba = gb_clf.predict_proba(X_valid)[:, 1]

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,gb_preds_proba, 0.5)

AUC:0.798
accuracy:0.726
recall:0.675
precision:0.121
specificity:0.729
prevalence:0.052
 


## XGB Model

In [25]:
xgb_clf = xgb.XGBClassifier(max_depth=3, learning_rate = 1.0, use_label_encoder = False,\
                            eval_metric = 'logloss')
xgb_clf.fit(X_train, y_train)

xgb_preds = xgb_clf.predict(X_valid)
xgb_preds_proba = xgb_clf.predict_proba(X_valid)[:, 1]

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,xgb_preds_proba, 0.5)

AUC:0.817
accuracy:0.722
recall:0.725
precision:0.126
specificity:0.722
prevalence:0.052
 


## Catboost Model

In [26]:
catb=CatBoostClassifier(iterations=200, depth=3, learning_rate=1.0, random_state = 111)
catb.fit(X_train, y_train)
catb_preds = catb.predict_proba(X_valid)[:, 1]

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,catb_preds, 0.5)

0:	learn: 0.5064841	total: 115ms	remaining: 22.8s
1:	learn: 0.4662750	total: 117ms	remaining: 11.6s
2:	learn: 0.4474312	total: 120ms	remaining: 7.9s
3:	learn: 0.4332415	total: 123ms	remaining: 6.04s
4:	learn: 0.4164106	total: 126ms	remaining: 4.92s
5:	learn: 0.3853243	total: 129ms	remaining: 4.17s
6:	learn: 0.3592484	total: 132ms	remaining: 3.63s
7:	learn: 0.3275688	total: 135ms	remaining: 3.23s
8:	learn: 0.3205659	total: 138ms	remaining: 2.92s
9:	learn: 0.3054361	total: 140ms	remaining: 2.67s
10:	learn: 0.2849862	total: 143ms	remaining: 2.46s
11:	learn: 0.2751173	total: 146ms	remaining: 2.29s
12:	learn: 0.2609338	total: 149ms	remaining: 2.15s
13:	learn: 0.2545338	total: 152ms	remaining: 2.02s
14:	learn: 0.2454883	total: 155ms	remaining: 1.91s
15:	learn: 0.2326937	total: 158ms	remaining: 1.81s
16:	learn: 0.2245675	total: 161ms	remaining: 1.73s
17:	learn: 0.2097930	total: 164ms	remaining: 1.66s
18:	learn: 0.2014676	total: 167ms	remaining: 1.59s
19:	learn: 0.1882524	total: 170ms	remainin

# Hyper Parameter Tuning

* From the above models we will choose two models for demonstration i.e. Random Forest, Decision Trees for hyper-parameter tuning.
* Generally you can pick up the top three models based on the 'AUC', 'Recall' or 'F1 score' score and tune them.

There are many techniques for hyper-parameter tuning:

* Random Search
* Grid Search
* Halving Grid Search(added recently in sklearn)

Special Note:
* It will take significant time to run Hyper Parameter Tuning 
* Timing will depend on available resources of server

In [27]:
recall_scoring = make_scorer(recall_score)

## Decision Tree - Hyper Parameter Tuning

In [28]:
dc_grid = {'max_features':['auto','sqrt'], # maximum number of features to use at each split
           'max_depth':range(1,11,1), # maximum depth of the tree
           'min_samples_split':range(2,10,2), # minimum number of samples to split a node
           'criterion':['gini','entropy']} # criterion for evaluating a split

dc_random = RandomizedSearchCV(estimator = dc_clf, param_distributions = dc_grid, 
                               n_iter = 20, cv = 2, scoring=recall_scoring,
                               verbose = 1, random_state = 111)

dc_random.fit(X_train, y_train)

dc_random.best_params_

dc_hp_preds = dc_random.best_estimator_.predict(X_valid)
dc_hp_preds_proba = dc_random.best_estimator_.predict_proba(X_valid)[:,1]
roc_auc_score(y_valid, dc_hp_preds_proba)

Fitting 2 folds for each of 20 candidates, totalling 40 fits


0.809486914600551

In [29]:
recall_score(y_valid, dc_hp_preds)

0.85

## Random Forest - Hyper Parameter Tuning

In [30]:
rf_grid = {'n_estimators':range(200,1000,200), # number of trees
           'max_features':['auto','sqrt'], # maximum number of features to use at each split
           'max_depth':range(1,11,1), # maximum depth of the tree
           'min_samples_split':range(2,10,2), # minimum number of samples to split a node
           'criterion':['gini','entropy']} # criterion for evaluating a split

rf_random = RandomizedSearchCV(estimator = rf_clf, param_distributions = rf_grid, 
                               n_iter = 20, cv = 2, scoring=recall_scoring,
                               verbose = 1, random_state = 111)

rf_random.fit(X_train, y_train)

rf_random.best_params_

rf_hp_preds = rf_random.best_estimator_.predict(X_valid)
rf_hp_preds_proba = rf_random.best_estimator_.predict_proba(X_valid)[:,1]
roc_auc_score(y_valid, rf_hp_preds_proba)

Fitting 2 folds for each of 20 candidates, totalling 40 fits


0.824862258953168

In [31]:
recall_score(y_valid, rf_hp_preds)

0.825

## XGBoost - Hyper Parameter Tuning

In [32]:
# xgb_grid = params = {
#         'min_child_weight': [1, 5, 8, 10],
#         'gamma': [0.5, 1, 1.5, 2, 5],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 0.9, 1.0],
#         'max_depth': [3, 4, 5]
#         } # criterion for evaluating a split

# xgb_random = GridSearchCV(estimator = xgb_clf, param_grid = xgb_grid, 
#                                cv = 2, scoring = recall_scoring,
#                                verbose = 1)

# xgb_random.fit(X_train, y_train)

# xgb_random.best_params_

# xgb_hp_preds = xgb_random.best_estimator_.predict(X_valid)
# xgb_hp_preds_proba = xgb_random.best_estimator_.predict_proba(X_valid)[:,1]
# roc_auc_score(y_valid, xgb_hp_preds_proba)

Fitting 2 folds for each of 720 candidates, totalling 1440 fits


0.8310261707988982

In [33]:
# recall_score(y_valid, xgb_hp_preds)

0.775